In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

%cd 'drive/MyDrive/Thesis_M'
!ls

Mounted at /content/drive
/content/drive/MyDrive/Thesis_M
 col_names.pkl					     ppi_network.gexf
 dGene_04-16-2013_2257.tsv			     PPI_V2_net.tsv
 features.pkl					     protein_network.txt
 labels.pkl					     Sir_M_Chapter_1_2.docx
'PANCANCER_ANOVA_Sun Nov 20 23_09_56 2022 (1).csv'


In [2]:
import pickle
import numpy as np
import xgboost as xgb
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import classification_report, confusion_matrix, recall_score, precision_score

# Load feature and label data from pickle files
with open('features.pkl', 'rb') as handle:
    X = pickle.load(handle)

with open('labels.pkl', 'rb') as handle:
    y = pickle.load(handle)

# Convert data to numpy arrays
X = np.array(X)
y = np.array(y)

In [3]:
# Encode labels
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_enc = le.fit_transform(y)


In [4]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# Define classifier

clf = BaggingClassifier()

# Define grid search parameters
param_grid = {
    'n_estimators': [5,8,10]   
}


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_enc, test_size=0.2, random_state=42)

# Perform grid search
grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X, y_enc)

# Print best parameters and score
print("Best parameters: ", grid_search.best_params_)
print("Best score: {:.2f}".format(grid_search.best_score_))

# Train and test the classifier with the best parameters
best_clf = grid_search.best_estimator_
best_clf.fit(X_train, y_train)
y_pred = best_clf.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test accuracy: {:.2f}".format(acc))


Best parameters:  {'n_estimators': 10}
Best score: 0.96
Test accuracy: 1.00
